In [44]:
import utils
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Flatten
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant 
from keras import Input

In [92]:
def CNN(data, wv_from_bin, batch_size=128, kernel_size=5):
    
    # Preparing Text Data and Label
    tweet, label = data['text'], data['sentiment']
    
    # Tokenizing Text
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(tweet)
    sequences = tokenizer.texts_to_sequences(tweet)

    # Preparing Embedding Dictionary
    word_index = tokenizer.word_index
    tweets_pad = pad_sequences(sequences, maxlen=max_length, padding='post')

    embedding_index = {}
    words = list(wv_from_bin.vocab.keys())
    curInd = 0
    for w in words:
        try:
            embedding_index[w] = wv_from_bin.word_vec(w)
            curInd += 1
        except KeyError:
            continue
            
    # Preparing Embedding Matrix
    embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, vec_size))
    words = []
    for word, i in word_index.items():
        words.append(word)
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            
    # Preparing Embedding Layer
    embedding_layer = Embedding(len(tokenizer.word_index) + 1,
                            vec_size,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)
    
    # Construction CNN Model Structure
    model = Sequential()
    model.add(embedding_layer)
    model.add(Dropout(0.4))
    model.add(Conv1D(filters, kernel_size, padding='valid', activation='relu', strides=1))
    model.add(Conv1D(300, kernel_size, padding='valid', activation='relu', strides=1))
    model.add(Conv1D(150, kernel_size, padding='valid', activation='relu', strides=1))
    model.add(Conv1D(75, kernel_size, padding='valid', activation='relu', strides=1))
    model.add(Flatten())
    model.add(Dense(600))
    model.add(Dropout(0.5))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('tanh'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
    filepath = "/Users/colinwan/Desktop/DataFest2020/checkpoint/4cnn-{epoch:02d}-{loss:0.3f}-{mean_squared_error:0.3f}-{val_loss:0.3f}-{val_mean_squared_error:0.3f}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=1, save_best_only=True, mode='min')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.000001)
    
    # Training Model
    model.fit(tweets_pad, label, batch_size=batch_size, epochs=8, validation_split=0.1, shuffle=True, callbacks=[checkpoint, reduce_lr])
    
    # Saving Trained Model
    model.save("/Users/colinwan/Desktop/DataFest2020/Models/model.h5")
    
    return model

In [93]:
CNN(tweets_pad, label)

Train on 12483 samples, validate on 1388 samples
Epoch 1/8
12483/12483 [==============================] - 107s 9ms/step - loss: 0.5431 - mean_squared_error: 0.5431 - val_loss: 0.4636 - val_mean_squared_error: 0.4636

Epoch 00001: loss improved from inf to 0.54309, saving model to /Users/colinwan/Desktop/DataFest2020/checkpoint/4cnn-01-0.543-0.543-0.464-0.464.hdf5
Epoch 2/8
12483/12483 [==============================] - 103s 8ms/step - loss: 0.4528 - mean_squared_error: 0.4528 - val_loss: 0.4325 - val_mean_squared_error: 0.4325

Epoch 00002: loss improved from 0.54309 to 0.45280, saving model to /Users/colinwan/Desktop/DataFest2020/checkpoint/4cnn-02-0.453-0.453-0.432-0.432.hdf5
Epoch 3/8
12483/12483 [==============================] - 111s 9ms/step - loss: 0.4113 - mean_squared_error: 0.4113 - val_loss: 0.4022 - val_mean_squared_error: 0.4022

Epoch 00003: loss improved from 0.45280 to 0.41132, saving model to /Users/colinwan/Desktop/DataFest2020/checkpoint/4cnn-03-0.411-0.411-0.402-0.4